
# Introduction to Feature Selection Project

This Jupyter notebook is part of a project focused on exploring various feature selection techniques to enhance the performance of machine learning models. The project uses the "Breast Cancer Wisconsin (Diagnostic)" dataset, applying methods like Mutual Information Feature Selection (MIFS), Correlation Feature Selection (CFS), and Sequential Forward Selection (SFS) among others, to identify the most significant features for accurate predictions.

The goal of this notebook is to provide a comprehensive analysis of these feature selection techniques, compare their effectiveness, and understand their impact on model performance. By the end of this notebook, we should have a clear understanding of which features are most important and why, as well as insights into the strengths and limitations of each feature selection method used.

This project is licensed under the MIT License - see the LICENSE file for details.

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo
from printScore import *
import pandas as pd

path = 'dataset/'
filename = 'breast-cancer-wisconsin.csv'

try:
	df = pd.read_csv(path + filename)
except FileNotFoundError:
	# fetch dataset 
	breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)
	
	# data (as pandas dataframes) 
	X = breast_cancer_wisconsin_diagnostic.data.features
	y = breast_cancer_wisconsin_diagnostic.data.targets
	
	# Create a Pandas DataFrame with the features and the target
	df = X.copy()
	df['target'] = y.copy()
	df.to_csv(path + filename, index=False)
	
df.info()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
enc.fit(df['target'].values.reshape(-1, 1))
df['target'] = enc.transform(df['target'].values.reshape(-1, 1))
df

# Discretization with ChiMerge

### Steps:

1. **Data Preparation**: Begin with continuous numerical data, and sort it in ascending order.

2. **Initial Binning**: Create initial bins, typically with a fixed number of data points or fixed width.

3. **Calculate Chi-Squared Statistic**: Calculate the chi-squared statistic for each pair of adjacent bins using observed and expected frequencies.

4. **Chi-Squared Test**: Apply a chi-squared test to decide whether to merge adjacent bins or keep them separate.

5. **Iterate**: Continue merging adjacent bins with chi-squared statistics below the threshold until you reach the desired number of bins or all chi-squared statistics exceed the threshold.

6. **Final Binning**: The boundaries of the bins represent the discrete intervals.

7. **Discretization**: Replace the continuous data with bin labels to indicate which bin each data point belongs to.

In [ ]:
from scorecardbundle.feature_discretization import ChiMerge as cm
from scorecardbundle.feature_encoding import WOE as woe

def chiMerge(df, X_features, y_features, max_intervals=10, min_intervals=2, decimal=None):
	trans_cm = cm.ChiMerge(max_intervals=max_intervals, min_intervals=min_intervals, decimal=decimal, output_dataframe=True)

	encoder = woe.WOE_Encoder()
	discreteDf = pd.DataFrame(
		encoder.fit_transform(trans_cm.fit_transform(df[X_features], df[y_features]), df[y_features]),
		columns=X_features, index=df.index
	)
	discreteDf[y_features] = df[y_features]

	uniqueValuesNumber = {feature : len(trans_cm.boundaries_[feature]) for feature in X_features}
	
	return discreteDf, uniqueValuesNumber, trans_cm.boundaries_

# Entropy 
![H(c) = - Sum(P(c) * log2(P(c)))](images/entropy.PNG)
# Joint Entropy
![H(C;F) = - Sum(P(c,f) * log2(P(c,f)))](images/jointEntropy.PNG)
# Conditional Entropy
![H(C|F) = H(C,F) - H(F)](images/conditionalEntropy.PNG)

In [ ]:
from mathFunctions import probability, jointProbabilities

# Cache Dictionary
entropyCache, jointEntropyCache = {}, {}

# H(c) = - Sum(P(c) * log2(P(c)))
def entropy(discreteDf, X_feature):
	if X_feature in entropyCache:
		return entropyCache[X_feature]
	
	prob = probability(discreteDf[X_feature].values)
	entropyCache[X_feature] = -np.sum(prob * np.log2(prob))

	return entropyCache[X_feature]

# H(C;F) = - Sum(P(c,f) * log2(P(c,f)))
def jointEntropy(discreteDf, X_feature, y_feature):
	# Create a key that doesn't depend on the order of the features
	tmp0, tmp1 = sorted([X_feature, y_feature]) # is a symmetric function
	key = f"{tmp0}-{tmp1}"#
	
	if key in jointEntropyCache:
		return jointEntropyCache[key]
	
	jp= jointProbabilities(discreteDf[X_feature].values, discreteDf[y_feature].values)
	jp = jp[jp > 0]
	jointEntropyCache[key] = - np.sum(jp * np.log2(jp))
	
	return jointEntropyCache[key]

# H(C|F) = H(C,F) - H(F)
def conditionalEntropy(discreteDf, X_feature, y_feature):
	return jointEntropy(discreteDf, X_feature, y_feature) - entropy(discreteDf, y_feature)

# Mutual Information
![mutualInformation](images/mutualInformation.PNG)

In [ ]:
# I_(f;c) = H(f) - H(f|c) Mutual Information function
def mutual_information(discreteDf, X_feature, y_feature):
	mi = entropy(discreteDf, X_feature) - conditionalEntropy(discreteDf, X_feature, y_feature)
	
	if mi < 0:
		print("Mutual Information error: ")
		print(f"I({X_feature};{y_feature}) = H({X_feature})-H({X_feature}, {y_feature}) = {entropy(discreteDf, X_feature):.3f} - {conditionalEntropy(discreteDf, X_feature, y_feature):.3f} = {mi:.5f}")
	
	return mi

#print("Finish: ", mutual_information(discreteDf, discreteDf.columns.tolist(), discreteDf.columns[-1]))

# Evaluation Function: Mutual Information Feature Selection (MIFS)

![MIFS_formula](images/MIFS_formula.PNG)

Where:
- f: feature in remaining features
- S: all the selected features
- C: the target feature (class)

**Reference**: Roberto Battiti, "Using Mutual Information for Selecting Features in Supervised Neural Net Learning", IEEE Transactions on Neural Networks, August 1994. DOI: [10.1109/72.298224](https://doi.org/10.1109/72.298224).

In [ ]:
import concurrent.futures
#from ipyparallel import Client
#c = Client(profile='default')

def mifs(discreteDf, selectedFeatures, feature, _class, beta=0.5):
	miFeatureClass = mutual_information(df, _class, feature) #I_(C;f) = H(C) - H(C|f)

	# I_(f;S) = Sum(I_(f;s)) for each s in S
	if not isinstance(selectedFeatures, list): # Check if y_features is a list
		selectedFeatures = [selectedFeatures]

	sum_mi_SelectedFeatures = 0
	with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor: # Use ThreadPoolExecutor to parallelize the calculations
		# Submit tasks for each combination of X_feature and y_feature
		futures = [executor.submit(mutual_information, discreteDf, feature, selectedFeature) for selectedFeature in selectedFeatures]
	for future in concurrent.futures.as_completed(futures): # Gather the results
		sum_mi_SelectedFeatures += future.result()
	#print(f"\nI_(f;S) = Sum(I_(f;s)) = {sum}\n-F = {feature}\n-S = {selectedFeatures}\n")
	
	# I_(C;f) - Beta * (Som(#I_(f;s)) for each s in S)
	result = miFeatureClass - beta * sum_mi_SelectedFeatures
	#print(f"I({_class};{feature})-Beta*(Som(I_({feature};s)) for each s in S) ={miFeatureClass:.3} - {beta} * {sum_mi_SelectedFeatures:.3f} = {result:.5f}")
	return result

# Evaluation Function: Correlation Feature Selection (CFS)

CFS is a filter-based method that evaluates the worth of a subset of features by considering the individual predictive ability of each feature along with the degree of redundancy between them.

![CSF_formula](images/CSF_formula.PNG)

In [ ]:
# Cache Dictionary
cfsCache = {}

def cfs(df, attributes, _class):
    # Create a key that doesn't depend on the order of the features
    if not isinstance(attributes, list): # If there is just one string 
        attributes = [attributes]
    else:
        attributes = sorted(attributes)

    key = f"{attributes}-{_class}"

    if key in cfsCache:
        return cfsCache[key]

    k = len(attributes)

    avgCorrAttributeClass = 0
    avgCorrAttributeAttribute = 0
    for attribute in attributes:
        # compute the average correlation between the attributes and the class
        avgCorrAttributeClass += np.abs(df[attribute].corr(df[_class]))
        # compute the average correlation between the attributes
        remainingAttributes = attributes.copy()
        remainingAttributes.remove(attribute)
        for attribute2 in remainingAttributes:
            avgCorrAttributeAttribute += np.abs(df[attribute].corr(df[attribute2]))

    avgCorrAttributeClass =  avgCorrAttributeClass / k
    avgCorrAttributeAttribute = avgCorrAttributeAttribute / (k * k)

    # compute the score of the testFeature
    cfsCache[key] = (k * avgCorrAttributeClass) / np.sqrt(k + k * (k - 1) * avgCorrAttributeAttribute)
    return cfsCache[key]

![variance](images/variance.PNG)

In [ ]:
# Cache Dictionary
varianceCache = {}

def variance(df, feature):
    # Use feature as key for the dictionary
    if feature in varianceCache:
        return varianceCache[feature]

    varianceCache[feature] = df[feature].var()
    return varianceCache[feature]

![correlation](images/correlation.PNG)

In [ ]:
# Cache Dictionary
correlationCache = {}    
def correlation(df, feature, target):
    # Create a key that doesn't depend on the order of the features
    tmp0, tmp1 = sorted([feature, target]) # is a symmetric function
    key = f"{tmp0}-{tmp1}"

    if key in correlationCache:
        return correlationCache[key]

    correlationCache[key] = np.abs(df[feature].corr(df[target]))
    return correlationCache[key]

## Sequential Forward Selection (SFS)

The SFS algorithm is a feature selection method that starts with a single attribute and incrementally adds attributes until the full set of attributes is reached. It is particularly effective when the optimal subset has only a few attributes. The process involves evaluating a large number of states initially, but as it progresses, the region examined by SFS becomes narrower since most of the attributes have already been selected.

![SFS](images/SFS.PNG)

## Implementation of the SFS Algorithm

In the following implementation of the SFS algorithm, we begin with a set of features, which can be either empty or contain initial attributes, and a list of remaining features, which includes all features except those already selected and the target feature. The algorithm operates through a series of iterations:

1. Compute the score of each remaining feature considering the selected features and the target feature.
2. Select the feature with the best score.
3. Add the selected feature to the list of selected features.
4. Remove the selected feature from the list of remaining features.

This process is repeated until the maximum number of iterations is reached or until there are no more remaining features to select from.

### Evaluation Functions for Feature Score

The score of each feature is computed using one of the following evaluation functions:

- Variance
- Correlation
- Correlation Feature Selection (CFS)
- Mutual Information Feature Selection (MIFS)

If multiple evaluation functions are given to the function, the score is based on the metrics order of priority given by the order of the metrics in the metrics list, so if the first score is the same for two features, the second score is used to determine the best feature and so on.

In [ ]:
from metrics import checkMetrics, createFeatureScore, compareFeatureScore
from utils import mifs_caching_init, mifs_caching_flush

allowedMetrics = ['variance', 'correlation', 'cfs', 'mifs']

# MIFS caching
discreteDf, entropyCache, jointEntropyCache, discreteDf_filename, entropyFilename, jointEntropyFilename = mifs_caching_init(path, filename, 'discrete.csv', 'entropy.csv', 'joint_entropy.csv')

def selection_sfs(df, selectedFeatures, remainingFeatures, target, metrics, maxIteration=np.inf, discreteDf=None, beta=0.5):
	metrics = checkMetrics(metrics, allowedMetrics)

	# Create the discrete data frame if needed
	mifs_cache = discreteDf is not None
	if allowedMetrics[3] in metrics and discreteDf is None:
		print("Warning: mifs cache not enabled")
		discreteDf, uniqueValuesNumber, featuresBoundaries = chiMerge(df, df.columns[:-1].tolist(), df.columns[-1], max_intervals=100, min_intervals=2)
		discreteDf.to_csv(path + discreteDf_filename, index=False)
		global entropyCache, jointEntropyCache
		entropyCache, jointEntropyCache = {}, {} # Cleaning Cache
		
		#print("Features Boundaries:\n", featuresBoundaries)
		#print("Features unique values:\n", pd.DataFrame.from_dict(uniqueValuesNumber,orient='index', columns=['Number of unique values']))
		#print("Discrete data frame:\n", discreteDf)		

	# Algorithm
	score = []
	counter = 0
	while counter < maxIteration and len(remainingFeatures) > 0:
		# initialize the best score and feature
		bestFeature = None
		bestScore = createFeatureScore(metrics)
		
		for feature in remainingFeatures:
			featureScore = createFeatureScore(metrics)# Initialize the feature score with the right metrics order
			
			# compute the variance score
			if allowedMetrics[0] in metrics: 
				featureScore[allowedMetrics[0]] = variance(df, feature) 
			# compute the correlation score
			if allowedMetrics[1] in metrics:
				featureScore[allowedMetrics[1]] = correlation(df, feature, target)
			# compute the cfs score
			if allowedMetrics[2] in metrics:
				featureScore[allowedMetrics[2]] = cfs(df, selectedFeatures + [feature], target)
			# compute the mifs score
			if allowedMetrics[3] in metrics:
				featureScore[allowedMetrics[3]] = mifs(df, selectedFeatures, feature, target, beta=beta)
				if mifs_cache:
					mifs_caching_flush(path, entropyCache, jointEntropyCache, entropyFilename, jointEntropyFilename)
			
			# check if the score is better than the best score considering all the metrics with order of priority
			if compareFeatureScore(featureScore, bestScore, metrics):
				bestScore = featureScore
				bestFeature = feature
			
		selectedFeatures.append(bestFeature)
		remainingFeatures.remove(bestFeature)
		score.append(bestScore)
		counter += 1
	
	return selectedFeatures, score, remainingFeatures

## Sequential Backward Selection (SBS)

The SBS algorithm is a feature selection method that begins with the full set of attributes and iteratively removes attributes until only a selected subset of features remains. It is particularly effective when the optimal subset has a large number of attributes. The process starts with the entire feature set and gradually narrows it down by eliminating less important features.

![SBS](images/SBS.PNG)

## Implementation of the SBS Algorithm

In the following implementation of the SBS algorithm, we start with the a set of remaining features and progressively reduce it through a series of iterations:

1. Compute for each remaining feature the score of all remaining feature except the evaluated one
2. Remove the feature with the lowest score.

This process is repeated until the maximum number of iterations is reached or until there is one remaining feature

### Evaluation Functions for Feature Score

The score of each feature is determined using one of the following evaluation functions:

- Variance
- Correlation
- Correlation Feature Selection (CFS)
- Mutual Information Feature Selection (MIFS)

If multiple evaluation functions are given to the function, the score is based on the metrics order of priority given by the order of the metrics in the metrics list, so if the first score is the same for two features, the second score is used to determine the best feature and so on.

In [ ]:
def selection_sbs(df, remainingFeatures, target, metrics, maxIteration=np.inf, discreteDf=None, beta=0.5):
    metrics = checkMetrics(metrics, allowedMetrics)

    # Create the discrete data frame if needed
    mifs_cache = discreteDf is not None
    if allowedMetrics[3] in metrics and discreteDf is None:
        print("Warning: mifs cache not enabled")
        discreteDf, uniqueValuesNumber, featuresBoundaries = chiMerge(df, df.columns[:-1].tolist(), df.columns[-1], max_intervals=100, min_intervals=2)
        discreteDf.to_csv(path + discreteDf_filename, index=False)
        global entropyCache, jointEntropyCache
        entropyCache, jointEntropyCache = {}, {} # Cleaning Cache

    #print("Features Boundaries:\n", featuresBoundaries)
    #print("Features unique values:\n", pd.DataFrame.from_dict(uniqueValuesNumber,orient='index', columns=['Number of unique values']))
    #print("Discrete data frame:\n", discreteDf)

    # Algorithm
    score = []
    eliminatedFeatures = []
    remainingFeatures = remainingFeatures.copy()
    while len(eliminatedFeatures) < maxIteration and len(remainingFeatures) > 1:
        # initialize the best score and feature
        worstFeature = None
        worstScore = createFeatureScore(metrics, positiveValue=True)

        for feature in remainingFeatures:
            featureScore = createFeatureScore(metrics) # Initialize the feature score with the right metrics order

            # Select all the features except the current feature
            testFeatures = remainingFeatures.copy()
            testFeatures.remove(feature)

            if allowedMetrics[0] in metrics:
                featureScore[allowedMetrics[0]] = variance(df, feature)
            if allowedMetrics[1] in metrics:
                featureScore[allowedMetrics[1]] = correlation(df, feature, target)
            if allowedMetrics[2] in metrics:
                featureScore[allowedMetrics[2]] = cfs(df, testFeatures, target)
            if allowedMetrics[3] in metrics:
                featureScore[allowedMetrics[3]] = mifs(df, testFeatures, feature, target, beta=beta)
                if mifs_cache:
                    mifs_caching_flush(path, entropyCache, jointEntropyCache, entropyFilename, jointEntropyFilename)

            # check if the score is worse than the worst score considering all the metrics with order of priority
            if compareFeatureScore(worstScore, featureScore, metrics):
                worstScore = featureScore
                worstFeature = feature

        remainingFeatures.remove(worstFeature)
        score.append(worstScore)
        eliminatedFeatures.append(worstFeature)

    return remainingFeatures, eliminatedFeatures, score

### Applying the SFS and SBS algorithms to the Breast Cancer Wisconsin (Diagnostic) Data Set

1. Initialize the selected features list
2. Initialize the remaining features list without the target
3. Get target name
4. Call the SFS function with maxIteration= 2
5. Call the SBS function with maxIteration= 3
6. Print the results

In [ ]:
target = df.columns[-1]

selectedFeatures, score, remainingFeatures = selection_sfs(df, [], df.columns[:-1].tolist(), target,
														   metrics=allowedMetrics[0], maxIteration=2)
remainingFeatures, eliminatedFeatures, worstScore = selection_sbs(df, remainingFeatures, target,
																  metrics=allowedMetrics[0], maxIteration= 3)

#printScoreDetails(selectedFeatures, score, remainingFeatures)
printFinalScore(selectedFeatures, score, eliminatedFeatures, worstScore)

## Bidirectional Selection (BDS)

The BDS algorithm is a search method that simultaneously explores the search space from both the initial state and the goal state. It is particularly effective in finding the shortest path between two states in a search graph. The process involves two simultaneous searches, one forward (SFS) from the initial state and one backward (SBS) from the goal state, with the goal of meeting in the middle.

![BDS](images/BDS.PNG)

## Implementation of the BDS Algorithm

In the following implementation of the SFS algorithm, we begin with a set of features, which can be either empty or contain initial attributes, and a list of remaining features, which includes all features except those already selected and the target feature. The algorithm operates through a series of iterations:

### Sequential Forward Selection (SFS):

1. Compute the score of each remaining feature considering the selected features and the target feature.
2. Select the feature with the best score.
3. Add the selected feature to the list of selected features.
4. Remove the selected feature from the list of remaining features.

### Sequential Backward Selection (SBS):

5. Compute for each remaining feature the score of all remaining features except the evaluated one.
6. Remove the feature with the lowest score.

This process is repeated until the maximum number of iterations is reached or until there are no more remaining features to select from. The goal of BDS is to minimize the search space by simultaneously exploring from both ends, reducing the time and resources needed to find a solution.

In [ ]:
def selection_bds(df, selectedFeatures, remainingFeatures, target, metrics, maxIteration=1, discreteDf=None, beta=0.5):
	metrics = checkMetrics(metrics, allowedMetrics)
	
	selectedFeaturesScore = []
	eliminatedFeaturesWorstScore = []
	eliminatedFeatures = []

	while(len(selectedFeatures) < maxIteration and len(remainingFeatures) > 0):
		selectedFeatures, selectedFeatureScore, remainingFeatures = selection_sfs(df, selectedFeatures, remainingFeatures, target, metrics, maxIteration= 1, discreteDf=discreteDf, beta=beta)
		remainingFeatures, eliminatedFeature, eliminatedFeatureWorstScore = selection_sbs(df, remainingFeatures, target, metrics, maxIteration= 1, discreteDf=discreteDf, beta=beta)

		selectedFeaturesScore.extend(selectedFeatureScore)
		maxIterationOverLoad = eliminatedFeature == []#It appends when |remainingFeatures| = 2
		if not maxIterationOverLoad:
			eliminatedFeaturesWorstScore.extend(eliminatedFeatureWorstScore)
			eliminatedFeatures.extend(eliminatedFeature)
		
	return selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore

### Bidirectional Selection with Variance as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_bds(df, [], df.columns[:-1].tolist(), target, metrics=allowedMetrics[0], maxIteration= 5)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

### Bidirectional Selection with Correlation as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_bds(df, [], df.columns[:-1].tolist(), target, metrics=allowedMetrics[1], maxIteration= 5)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

### Bidirectional Selection with Correlation Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_bds(df, [], df.columns[:-1].tolist(), target, metrics=allowedMetrics[2], maxIteration= 5)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

### Bidirectional Selection with Mutual Information Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_bds(df, [], df.columns[:-1].tolist(), target, metrics=allowedMetrics[3], maxIteration= 5, discreteDf=discreteDf)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

## Plus-l-Minus-r Selection (LRS)

The Plus-l-Minus-r Selection method is like the Bidirectional Selection (BDS) algorithm, but it allows the forward and backward searches to be performed with different step sizes.The l parameter controls the number of features added in each iteration of the forward search (SFS), while the r parameter controls the number of features removed in each iteration of the backward search (SBS). 

![Plus-l-Minus-r Selection](images/LRS.PNG)

### Implementation of the Plus-l-Minus-r Selection Algorithm

In the following implementation of the Plus-l-Minus-r Selection algorithm, we begin with a set of features. This set can initially be empty or contain the starting attributes. We also maintain a list of remaining features, which includes all features except those already selected and the target feature. The algorithm operates through a series of iterations:

#### Sequential Forward Selection (SFS):

1. Calculate the score for each remaining feature, considering the selected features and the target feature.
2. Choose the feature with the highest score.
3. Add the selected feature to the list of chosen features.
4. Remove the selected feature from the list of remaining features.
5. Repeat steps 1-4 until l features have been selected.

#### Sequential Backward Selection (SBS):

6. Calculate the score for each remaining feature, considering all remaining features except the one being evaluated.
7. Remove the feature with the lowest score.
8. Repeat steps 6-7 until r features have been removed.

This process is repeated until the maximum number of iterations is reached or until there are no more remaining features to select from. The goal of Plus-l-Minus-r Selection is to minimize the search space by simultaneously exploring from both ends, reducing the time and resources needed to find a solution.

In [ ]:
# Implementation of the LRS algorithm using the SFS and SBS function
def selection_lrs(df, selectedFeatures, remainingFeatures, target, metrics, l=1, r=1, maxIteration=1, discreteDf=None, beta=0.5):
	metrics = checkMetrics(metrics, allowedMetrics)
	
	selectedFeaturesScore = []
	eliminatedFeaturesWorstScore = []
	eliminatedFeatures = []

	while(len(selectedFeatures) < maxIteration and len(remainingFeatures) > 0):
		selectedFeatures, selectedFeatureScore, remainingFeatures = selection_sfs(df, selectedFeatures, remainingFeatures, target, metrics, maxIteration= l, discreteDf=discreteDf, beta=beta)
		remainingFeatures, eliminatedFeature, eliminatedFeatureWorstScore = selection_sbs(df, remainingFeatures, target, metrics, maxIteration= r, discreteDf=discreteDf, beta=beta)

		selectedFeaturesScore.extend(selectedFeatureScore)
		maxIterationOverLoad = eliminatedFeature == []#It appends when |remainingFeatures| = 2
		if not maxIterationOverLoad:
			eliminatedFeaturesWorstScore.extend(eliminatedFeatureWorstScore)
			eliminatedFeatures.extend(eliminatedFeature)
	
	return selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore

## Plus-l-Minus-r Selection with Variance as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_lrs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[0], maxIteration= 5, l=2, r=1)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

## Plus-l-Minus-r Selection with Correlation as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_lrs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[1], maxIteration= 5, l=2, r=1)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

## Plus-l-Minus-r Selection with Correlation Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_lrs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[2], maxIteration= 5, l=2, r=1)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

## Plus-l-Minus-r Selection with Mutual Information Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures, eliminatedFeaturesWorstScore = selection_lrs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[3], maxIteration= 5, discreteDf=discreteDf, l=2, r=1)

printFinalScore(selectedFeatures, selectedFeaturesScore, eliminatedFeatures, eliminatedFeaturesWorstScore)

## Sequential Forward Floating Selection (SFFS)

The Sequential Forward Floating Selection (SFFS) method is a feature selection technique that combines elements of forward selection and backward elimination. SFFS aims to find an optimal subset of features by iteratively adding and removing features based on their performance scores.

![SFFS](images/SFFS.PNG)

### Implementation of the Sequential Forward Floating Selection Algorithm

SFFS (Sequential Forward Floating Selection) yielding a subset of d features, with optional search-restricting parameter ∆ ∈ [0, D − d]:
1. Start with X0 = ∅, k = 0.
2. Xk+1 = ADD(Xk), k = k + 1.
3. Repeat Xk−1 = REMOVE(Xk), k = k − 1 as long as it improves solutions already known for the lower k.
4. If k < d + ∆ go to 2.

![SFFS2](images/SFFS_algorithm.PNG)

The goal of SFFS is to gradually build a feature subset that optimizes a specific evaluation criterion while allowing for the removal of previously selected features if it benefits the overall performance.

**Reference**: Somol, Petr & Novovicova, Jana & Pudil, Pavel. "Efficient Feature Subset Selection and Subset Size Optimization", in book: "Pattern Recognition Recent Advances", February 2010. DOI: [10.5772/9356](https://doi.org/10.5772/9356).

In [ ]:
def selection_sffs(df, selectedFeatures, remainingFeatures, target,
                   metrics, maxIteration=1, discreteDf=None, beta=0.5):
    metrics = checkMetrics(metrics, allowedMetrics)
    selectedFeaturesScore = []
    removedFeatures = []

    while len(selectedFeatures) < maxIteration and remainingFeatures:
        # Forward step (add best feature)
        selectedFeatures, featScore, remainingFeatures = selection_sfs(
            df, selectedFeatures, remainingFeatures, target,
            metrics, maxIteration=1, discreteDf=discreteDf, beta=beta)
        selectedFeaturesScore.extend(featScore)

        # Floating backward steps
        improved = True
        while improved and len(selectedFeatures) > 1:
            newSelected, eliminated, elimScore = selection_sbs(
                df, selectedFeatures, target,
                metrics, maxIteration=1, discreteDf=discreteDf, beta=beta)
            if eliminated:
                selectedFeatures = newSelected
                remainingFeatures.extend(eliminated)
                removedFeatures.extend(eliminated)
                selectedFeaturesScore.extend(elimScore)
            else:
                improved = False

    return selectedFeatures, selectedFeaturesScore, remainingFeatures, removedFeatures

## Sequential Forward Floating Selection (SFFS) with variance as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures = selection_sffs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[0], maxIteration= 2)

printScoreDetails(selectedFeatures, selectedFeaturesScore, eliminatedFeatures)

## Sequential Forward Floating Selection (SFFS) with correlation as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures = selection_sffs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[1], maxIteration= 2)

printScoreDetails(selectedFeatures, selectedFeaturesScore, eliminatedFeatures)

## Sequential Forward Floating Selection (SFFS) with Correlation Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures = selection_sffs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[2], maxIteration= 2)

printScoreDetails(selectedFeatures, selectedFeaturesScore, eliminatedFeatures)

## Sequential Forward Floating Selection (SFFS) with Mutual Information Feature Selection as evaluation function

In [ ]:
selectedFeatures, selectedFeaturesScore, remainingFeatures, eliminatedFeatures = selection_sffs(df, [], df.columns[:-1].tolist(), df.columns[-1], metrics=allowedMetrics[3], maxIteration= 2)

printScoreDetails(selectedFeatures, selectedFeaturesScore, eliminatedFeatures)

## Sequential Backward Floating Selection (SBFS)

Sequential Backward Floating Selection) yielding a subset of d features, with optional search-restricting parameter Δ ∈ [0, d]:

![SFFS](images/SBFS_algorithm.PNG)
